In [2]:
import os
import zipfile
from shutil import copyfile
import time
from datetime import timedelta
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from tensorflow.keras.models import load_model
from tensorflow.keras import models

In [4]:
# source https://colab.research.google.com/github/lmoroney/mlday-tokyo/blob/master/Lab6-Cats-v-Dogs.ipynb#scrollTo=7v55rWlQehzL

# Note: This is a very large dataset and will take time to download

!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
try:
    os.mkdir('/tmp/cats-v-dogs')
    os.mkdir('/tmp/cats-v-dogs/training')
    os.mkdir('/tmp/cats-v-dogs/testing')
    os.mkdir('/tmp/cats-v-dogs/training/cats')
    os.mkdir('/tmp/cats-v-dogs/training/dogs')
    os.mkdir('/tmp/cats-v-dogs/testing/cats')
    os.mkdir('/tmp/cats-v-dogs/testing/dogs')
except OSError:
    pass

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .85
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [ ]:
Traininglen = len(os.listdir('/tmp/cats-v-dogs/training/cats/')) + len(os.listdir('/tmp/cats-v-dogs/training/dogs/'))
print(Traininglen)

# Total of images are 25000, so 85% should be 21250

In [ ]:
# Model 1 - 1 conv layer

model1 = Sequential()

# Conv Layer 1
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model1.add(MaxPooling2D((2, 2)))

# flatten
model1.add(Flatten())

# Fully connected Lyaer
model1.add(Dense(128, activation='relu'))

# Output layer
model1.add(Dense(1, activation='sigmoid'))

# compile
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])


In [ ]:
# Model 2 - 2 conv layers
model2 = Sequential()

# Conv Layer 1
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model2.add(MaxPooling2D((2, 2)))

# Conv Layer 2
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))

model2.add(Flatten())

# Fully connected Lyaers
model2.add(Dense(128, activation='relu'))

# Output layer
model2.add(Dense(1, activation='sigmoid'))

# compile
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# Model 3 - 5 conv layers - No Batchnormalization and No dropout
model3_NBND = Sequential()

# Conv Layer 1
model3_NBND.add(Conv2D(64, (3, 3), activation='relu', input_shape=(128,128,3)))
model3_NBND.add(MaxPooling2D((2, 2)))

# Conv Layer 2
model3_NBND.add(Conv2D(128, (3, 3), activation='relu'))
model3_NBND.add(MaxPooling2D((2, 2)))

# Conv Layer 3
model3_NBND.add(Conv2D(256, (3, 3), activation='relu'))
model3_NBND.add(MaxPooling2D((2, 2)))

# Conv Layer 4
model3_NBND.add(Conv2D(256, (3, 3), activation='relu'))
model3_NBND.add(MaxPooling2D((2, 2)))

# Conv Layer 5
model3_NBND.add(Conv2D(512, (3, 3), activation='relu'))
model3_NBND.add(MaxPooling2D(pool_size=(2, 2)))

model3_NBND.add(Flatten())


# Fully connected Lyaers
model3_NBND.add(Dense(units=1024, activation='relu'))

model3_NBND.add(Dense(units=256, activation='relu'))

# Output layer
model3_NBND.add(Dense(units=1, activation='sigmoid'))

# compile
model3_NBND.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [ ]:
# Agian Model 3 - 5 conv layers - but with Batchnormalization and dropout

model3_WBD = Sequential()

# Conv Layer 1
model3_WBD.add(Conv2D(64, (3, 3), input_shape=(128,128,3)))
model3_WBD.add(BatchNormalization(axis=-1))
model3_WBD.add(Activation('relu'))
model3_WBD.add(MaxPooling2D(pool_size=(2, 2)))

# Conv Layer 2
model3_WBD.add(Conv2D(128, (3, 3)))
model3_WBD.add(BatchNormalization(axis=-1))
model3_WBD.add(Activation('relu'))
model3_WBD.add(MaxPooling2D((2, 2)))

# Conv Layer 3
model3_WBD.add(Conv2D(256, (3, 3)))
model3_WBD.add(BatchNormalization(axis=-1))
model3_WBD.add(Activation('relu'))
model3_WBD.add(MaxPooling2D((2, 2)))

# Conv Layer 4
model3_WBD.add(Conv2D(256, (3, 3)))
model3_WBD.add(BatchNormalization(axis=-1))
model3_WBD.add(Activation('relu'))
model3_WBD.add(MaxPooling2D((2, 2)))

# Conv Layer 5
model3_WBD.add(Conv2D(512, (3, 3)))
model3_WBD.add(BatchNormalization(axis=-1))
model3_WBD.add(Activation('relu'))
model3_WBD.add(MaxPooling2D(2, 2))

# flatten
model3_WBD.add(Flatten())

# full connection w/ dropout  
model3_WBD.add(Dense(units=1024))
model3_WBD.add(Activation('relu'))
model3_WBD.add(Dropout(0.5))

model3_WBD.add(Dense(units=256))
model3_WBD.add(Activation('relu')) 
model3_WBD.add(Dropout(0.5))

# output layer
model3_WBD.add(Dense(units=1))
model3_WBD.add(Activation('sigmoid'))

In [ ]:
ModelList = [model1, model2, model3_NBND, model3_NBND]

In [ ]:
# adjust learning rate for plateaus
# we will start with a higher learning rate and /
# set it to reduce by half if our loss does not improve for 3 epochs, /
#  with a minimum learning rate of 0.00001.
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, 
                                            verbose=2, factor=0.5, min_lr=0.00001)

In [ ]:
# early-stopping conditions for the training of our model: \
# if the loss does not improve by at least 1e-10 for 25 epochs, then training will stop
early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-10, 
                               patience=25, restore_best_weights=True)

In [ ]:
# Image processing, data augmentation

Batch = 256

TRAINING_DIR = "/tmp/cats-v-dogs/training/"

train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=Batch,
                                                    class_mode='binary',
                                                    target_size=(128, 128))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"

validation_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=Batch,
                                                              class_mode='binary',
                                                              target_size=(128, 128))

In [ ]:
count = 1
for model in models:
    if count == 1:
        modelname = "model1.h5"
    if count == 2:
        modelname = "model2.h5"
    if count == 3:
        modelname = "model3_NBND.h5"
    if count == 4:
        modelname = "model3WD.h5"

    # save Model While Training
    best_model = ModelCheckpoint(modelname, monitor='val_acc', verbose=0, 
                             save_best_only=True, mode='max')
    CSV = CSVLogger(modelname, separator=",", append=False)

    time_initial = time.time() # how much time took
    history = model.fit(train_generator, epochs=100, steps_per_epoch= Traininglen/Batch,
                    validation_data=validation_generator, validation_steps=6,
                    callbacks=[early_stopping, learning_rate_reduction, best_model, CSV])
                    
    time_elapsed = time.time() - time_initial